In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mp
from qutip import *
from numpy import pi

In [13]:
# energy levels
N = 3 # Qutrit
id = qeye(N) # identity matrix

# Annihilation operator for the transmon
a = destroy(N)

# Set rotating frame frequency (should be close to qubit frequencies for fast simulations)
rotating_frame = 2*np.pi*5.0 * 1000

# Qubit frequency (0->1) in MHz
omega = 2*np.pi*5.708390 * 1000

# Self-Kerr coefficient (anharmonicity) in MHz
alpha = -2*np.pi*261.081457

# Transmon Hamiltonian
H_qubit = (omega-rotating_frame) * a.dag() * a + alpha * pow(a.dag(),2) * pow(a,2)

# Drive Hamiltonian (since we have drag pulses, we need both x and y drive)
H_drive_x = a.dag() + a
H_drive_y = -1j*(a.dag() - a)

def H_drive_coeff(t, drive_freq, y, args):
    sigma = 0.010 # Gaussian width in μs
    L = 0.050 # Gate time in μs
    B = args['amp']*np.pi/0.050 # Amplitude, needs to be optimized to get perfect pi pulse or pi/2 pulse
    q = args['qscale'] # DRAG-parameter, needs to be optimized to get no phase errors in a pi/2 pulse
    # E(t)
    E = B * np.exp(-pow(t-0.5*L, 2) / (2*pow(sigma,2)))
    if y: # coefficients for H_drive_y
        I = q * (t-0.5*L) / sigma * E
        Q = E
    else: # coefficients for H_drive_x
        I = E
        Q = q * (t-0.5*L) / sigma * E
    return I*np.cos(drive_freq * t) + Q*np.sin(drive_freq * t) 

# total Hamiltonian
H = [H_qubit,
     [H_drive_x,lambda t,args: H_drive_coeff(t, omega-rotating_frame, False, args)],
     [H_drive_y,lambda t,args: H_drive_coeff(t, omega-rotating_frame, True, args)]
    ]

In [15]:
# initial state vacuum
vac = basis(N,0)

t_total = 0.05

tlist = np.linspace(0,t_total,200)
opt = Options(nsteps=5000, atol=1e-8, rtol=1e-8)
args = {'amp': 0.5, 'qscale': 0.5}
result = mesolve(H, vac, tlist, [], [], options=opt, args=args)

In [17]:
final_state = result.states[-1] # final state

# unitary matrix to into transmon frame
rotate_U = Qobj([[1,0,0],
                 [0,np.exp(1j*(omega-rotating_frame)*t_total),0],
                 [0,0,np.exp(1j*(omega+alpha-rotating_frame)*t_total)]])

# rotated transmon state 
final_state = rotate_U*final_state

# project onto qubit subspace
qubit_state = (basis(2,0)*basis(3,0).dag() + basis(2,1)*basis(3,1).dag())*final_state
qubit_state = qubit_state / np.sqrt( (qubit_state.dag()*qubit_state)[0,0] )

# target state
target_state = rx(np.pi/2)*basis(2,0)

fid = fidelity(target_state, qubit_state)